In [1]:
# Visualize prelim_CI_data.xlsx
import pandas as pd
import numpy as np
import altair as alt

/home/sphsc/cse583/.blap/lib/python3.12/site-packages/numpy/_core/getlimits.py:552: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
df = pd.DataFrame(pd.read_csv('prelim_CI_data.csv'))
df

,Frequency,Amplitude,95% CI
0,55,115,"1.473928002396284, 2.008154796878246"
1,55,110,"0.8573880720694008, 1.3385235026344884"
2,55,105,"0.05739640062737363, 0.5116856983652559"
3,55,100,"-0.18618362292523702, 0.18179032878340542"
4,55,95,"-0.1184828607978683, 0.10077856684786743"
5,55,90,"-0.10215328691028783, 0.030957263160200508"
6,70,90,"-0.049966512010400435, 0.16282547978757833"
7,70,95,"0.10112030610532674, 0.5025200837537513"
8,70,100,"0.6396918483607472, 1.1569442391301974"
9,70,105,"0.9661967540332859, 1.3928917693731726"


In [ ]:
df[['CI_2.5%', 'CI_97.5%']] = df['95% CI'].str.split(',', expand=True)
# Convert the split columns from strings to float
df['CI_2.5%'] = df['CI_2.5%'].astype(float)
df['CI_97.5%'] = df['CI_97.5%'].astype(float)
df

,Frequency,Amplitude,95% CI,CI_2.5%,CI_97.5%,Midpoint
0,55,115,"1.473928002396284, 2.008154796878246",1.473928,2.008155,2.478005
1,55,110,"0.8573880720694008, 1.3385235026344884",0.857388,1.338524,1.526650
2,55,105,"0.05739640062737363, 0.5116856983652559",0.057396,0.511686,0.313239
3,55,100,"-0.18618362292523702, 0.18179032878340542",-0.186184,0.181790,-0.095288
4,55,95,"-0.1184828607978683, 0.10077856684786743",-0.118483,0.100779,-0.068094
5,55,90,"-0.10215328691028783, 0.030957263160200508",-0.102153,0.030957,-0.086675
6,70,90,"-0.049966512010400435, 0.16282547978757833",-0.049967,0.162825,0.031446
7,70,95,"0.10112030610532674, 0.5025200837537513",0.101120,0.502520,0.352380
8,70,100,"0.6396918483607472, 1.1569442391301974",0.639692,1.156944,1.218164
9,70,105,"0.9661967540332859, 1.3928917693731726",0.966197,1.392892,1.662643


In [51]:
# Manually recreate the dataframe from the values
data_dict = {
    'Frequency': df['Frequency'].tolist(), #.tolist() removes index information, convert to pure python list, 
    'Amplitude': df['Amplitude'].tolist(),
    'CI_lower': df['CI_2.5%'].tolist(),
    'CI_upper': df['CI_97.5%'].tolist()
}

df_new = pd.DataFrame(data_dict)
df_new['Midpoint'] = (df_new['CI_lower'] + df_new['CI_upper']) / 2
df_new

,Frequency,Amplitude,CI_lower,CI_upper,Midpoint
0,55,115,1.473928,2.008155,1.741041
1,55,110,0.857388,1.338524,1.097956
2,55,105,0.057396,0.511686,0.284541
3,55,100,-0.186184,0.181790,-0.002197
4,55,95,-0.118483,0.100779,-0.008852
5,55,90,-0.102153,0.030957,-0.035598
6,70,90,-0.049967,0.162825,0.056429
7,70,95,0.101120,0.502520,0.301820
8,70,100,0.639692,1.156944,0.898318
9,70,105,0.966197,1.392892,1.179544


In [66]:
# Add a zero column to your data
df_new['zero'] = 0

# Create area layer
area = alt.Chart(df_new).mark_area(opacity=0.3).encode(
    x=alt.X('Amplitude:Q', title='Stimulus Amplitude'),
    y=alt.Y('CI_lower:Q', title='95% Confidence Interval'),
    y2='CI_upper:Q',
    color=alt.Color('Frequency:O',title = 'Frequency (Hz)', scale=alt.Scale(scheme='viridis'))
)

# Create y=0 line
zero_line = alt.Chart(df_new).mark_rule(color='black', strokeWidth=.5, strokeDash=[5, 5], opacity=0.3).encode(
    y=alt.datum(0)
)

# Create midpoint line
midpoint_line = alt.Chart(df_new).mark_line(strokeWidth=2).encode(
    x='Amplitude:Q',
    y='Midpoint:Q',
    color=alt.Color('Frequency:O', scale=alt.Scale(scheme='viridis'))
)

# Create circles
circles = alt.Chart(df_new).mark_circle(size=60).encode(
    x='Amplitude:Q',
    y='Midpoint:Q',
    color=alt.Color('Frequency:O', scale=alt.Scale(scheme='viridis')),
    tooltip = ['Amplitude','Frequency','CI_lower','CI_upper']
)

# Layer then facet
chart = (area + zero_line + midpoint_line + circles).properties(
    width=300,
    height=300
).facet(
    column=alt.Column('Frequency:N', title='Frequency (Hz)')
).configure_axis(
    labelFontSize=13,
    titleFontSize=14
).configure_header(
    labelFontSize=18,
    titleFontSize=16  
).configure_legend(
    labelFontSize=13,
    titleFontSize=16
)

chart

alt.FacetChart(...)

In [48]:
# Create area layer
area = alt.Chart(df_new).mark_area(opacity=0.3).encode(
    x=alt.X('Amplitude:Q', title='Stimulus Amplitude'),
    y=alt.Y('CI_lower:Q', title='95% Confidence Interval', scale=alt.Scale(domain=[-0.5, 2.5])),
    y2='CI_upper:Q',
    color=alt.Color('Frequency:O', scale=alt.Scale(scheme='viridis'))
)

# Create horizontal line using mark_rule without x encoding
line = alt.Chart(df_new).mark_rule(color='black', strokeWidth=0.5, strokeDash=[5,5], opacity=0.5).encode(
    y=alt.datum(0)
)

# Layer then facet
chart = (area + line).properties(
    width=300,
    height=300
).facet(
    column='Frequency:N'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_header(
    labelFontSize=16,
    titleFontSize=16  
).configure_legend(
    labelFontSize=13,
    titleFontSize=16
)

chart

alt.FacetChart(...)